In [178]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from IPython.display import display
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [179]:
df = pd.read_csv('./train.csv')

In [180]:
df.shape

(2051, 81)

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
Id                 2051 non-null int64
PID                2051 non-null int64
MS SubClass        2051 non-null int64
MS Zoning          2051 non-null object
Lot Frontage       1721 non-null float64
Lot Area           2051 non-null int64
Street             2051 non-null object
Alley              140 non-null object
Lot Shape          2051 non-null object
Land Contour       2051 non-null object
Utilities          2051 non-null object
Lot Config         2051 non-null object
Land Slope         2051 non-null object
Neighborhood       2051 non-null object
Condition 1        2051 non-null object
Condition 2        2051 non-null object
Bldg Type          2051 non-null object
House Style        2051 non-null object
Overall Qual       2051 non-null int64
Overall Cond       2051 non-null int64
Year Built         2051 non-null int64
Year Remod/Add     2051 non-null int64
Roof Style         20

In [182]:
df.describe()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2051.000000,2.051000e+03,2051.000000,1721.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2029.000000,...,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000
mean,1474.033642,7.135900e+08,57.008776,69.055200,10065.208191,6.112140,5.562165,1971.708922,1984.190151,99.695909,...,93.833740,47.556802,22.571916,2.591419,16.511458,2.397855,51.574354,6.219893,2007.775719,181469.701609
std,843.980841,1.886918e+08,42.824223,23.260653,6742.488909,1.426271,1.104497,30.177889,21.036250,174.963129,...,128.549416,66.747241,59.845110,25.229615,57.374204,37.782570,573.393985,2.744736,1.312014,79258.659352
min,1.000000,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,753.500000,5.284581e+08,20.000000,58.000000,7500.000000,5.000000,5.000000,1953.500000,1964.500000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129825.000000
50%,1486.000000,5.354532e+08,50.000000,68.000000,9430.000000,6.000000,5.000000,1974.000000,1993.000000,0.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,162500.000000
75%,2198.000000,9.071801e+08,70.000000,80.000000,11513.500000,7.000000,6.000000,2001.000000,2004.000000,161.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2930.000000,9.241520e+08,190.000000,313.000000,159000.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,1424.000000,547.000000,432.000000,508.000000,490.000000,800.000000,17000.000000,12.000000,2010.000000,611657.000000


In [183]:
display(df.head())

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [184]:
#df['Street'].value_counts()
df['Year Built'].value_counts()
#df['Land Contour'].value_counts()
#df['Functional'].value_counts()
df['Neighborhood'].value_counts()
#df['Land Slope'].value_counts()
df['MS SubClass'].value_counts()
df['MS Zoning'].value_counts()
#df['Alley'].value_counts() # Mostly null
df['Lot Shape'].value_counts()
#df['Utilities'].value_counts()
df['Lot Config'].value_counts()
df['Condition 1'].value_counts()
#df['Condition 2'].value_counts()a
df['Bldg Type'].value_counts()
df['House Style'].value_counts()
df['Year Built'].value_counts() # Non-ordinal
df['Year Remod/Add'].value_counts() # Non-ordinal
#df['Roof Style'].value_counts()
#df['Roof Matl'].value_counts()
df['Exterior 1st'].value_counts()
#df['Exterior 2nd'].value_counts()
df['Mas Vnr Type'].value_counts() # Possible interaction with Vnr Area
df['Exter Qual'].value_counts()
df['Exter Cond'].value_counts() # Possible interaction with Exter Qual
df['Foundation'].value_counts()
df['Bsmt Qual'].value_counts()
#df['Bsmt Cond'].value_counts() # Maybe interacts with Qual?
df['Bsmt Exposure'].value_counts() # Possible interation with other bsmt stuff
df['BsmtFin Type 1'].value_counts()
#df['BsmtFin Type 2'].value_counts() # Not many with 2 basements
#df['Heating'].value_counts() # Too homogenous
df['Heating QC'].value_counts()
df['Central Air'].value_counts() # Maybe... Only one column...
#df['Electrical'].value_counts() # Too homogenous
df['Kitchen Qual'].value_counts() # Check interaction with other kitchen stuff
#df['Functional'].value_counts() # Too homogenous
#df['Fireplace Qu'].value_counts() # Mostly null
df['Garage Type'].value_counts()
#df['Garage Yr Blt'].value_counts() # Non ordinal, though newer is better I guess
df['Garage Finish'].value_counts()
#df['Garage Qual'].value_counts() # Too homogenous
#df['Garage Cond'].value_counts() # Too homogenous
df['Paved Drive'].value_counts() # Pretty homogenous but also only adds two columns...
#df['Pool QC'].value_counts() # Mostly null values
#df['Fence'].value_counts() # Mostly null values
##df['Misc Feature'].value_counts() # Mostly null but always adds value to property
##df['Misc Val'].value_counts() # Interacts with above probably?
df['Mo Sold'].value_counts() # Non ordinal. Is it even relevant?
df['Yr Sold'].value_counts() # Non ordinal; prices do change through the years
df['Sale Type'].value_counts() # No idea what it means but it's worth checking out

WD       1781
New       160
COD        63
ConLD      17
CWD        10
ConLI       7
ConLw       5
Oth         4
Con         4
Name: Sale Type, dtype: int64

In [185]:
# Drop columns composed largely of null values. Still on the fence about
# the Misc Features category
df = df.drop(columns=['Alley', 'Pool QC', 'Fence', 'Misc Feature', 'Fireplace Qu'])
# Drop non-numeric columns that are not good dummy candidates
df.drop(columns=['Street', 'Land Contour', 'Functional', 'Land Slope',
                     'Utilities', 'Condition 2', 'Roof Style', 'Roof Matl',
                     'Exterior 2nd', 'Bsmt Cond', 'BsmtFin Type 2', 'Heating',
                     'Electrical', 'Functional', 'Garage Yr Blt', 'Garage Qual',
                     'Garage Cond'], inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(30,30))
sns.heatmap(df.corr(), annot=True);

In [101]:
# Drop remaining columns with low correlation to target before scaling, since
# correlation is independent of scale and origin
df.drop(columns=['Pool Area', 'Screen Porch', '3Ssn Porch', 'Enclosed Porch',
                'Kitchen AbvGr', 'Bedroom AbvGr', 'Bsmt Half Bath',
                'Low Qual Fin SF', 'BsmtFin SF 2', 'PID'], inplace=True)

In [102]:
# Generate dummies with potential valuee
df = pd.get_dummies(df, columns=['Year Built', 'Neighborhood', 'MS SubClass',
               'MS Zoning', 'Lot Shape', 'Lot Config', 'Condition 1',
               'Bldg Type', 'House Style', 'Year Remod/Add',
               'Exterior 1st', 'Mas Vnr Type', 'Exter Qual', 'Exter Cond',
               'Foundation', 'Bsmt Qual', 'Bsmt Exposure', 'BsmtFin Type 1',
               'Heating QC', 'Central Air', 'Kitchen Qual', 'Garage Type',
               'Garage Finish', 'Paved Drive', 'Mo Sold', 'Yr Sold',
               'Sale Type'], drop_first=True)

In [11]:
# Ensure presence of only numerical datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Columns: 462 entries, Id to Sale Type_WD 
dtypes: float64(8), int64(15), uint8(439)
memory usage: 1.2 MB


In [12]:
# Find location of null vals
df.isnull().sum().sort_values(ascending=False)

Lot Frontage            330
Mas Vnr Area             22
Bsmt Full Bath            2
Garage Area               1
Garage Cars               1
BsmtFin SF 1              1
Bsmt Unf SF               1
Total Bsmt SF             1
Sale Type_WD              0
Neighborhood_Somerst      0
Neighborhood_SawyerW      0
Neighborhood_Sawyer       0
Neighborhood_SWISU        0
Neighborhood_NoRidge      0
Neighborhood_OldTown      0
Neighborhood_NridgHt      0
Neighborhood_Timber       0
Neighborhood_NWAmes       0
Neighborhood_NPkVill      0
Neighborhood_NAmes        0
Neighborhood_StoneBr      0
MS SubClass_40            0
Neighborhood_Veenker      0
MS SubClass_30            0
Neighborhood_MeadowV      0
MS SubClass_45            0
MS SubClass_50            0
MS SubClass_60            0
MS SubClass_70            0
MS SubClass_75            0
                       ... 
Year Remod/Add_1973       0
Year Built_2008           0
Year Built_2007           0
Year Built_2006           0
Year Built_1992     

In [103]:
# Drop Lot Frontage for nulls
df.drop(columns=['Lot Frontage'], inplace=True)

In [117]:
# Fill all remaining null rows with 0s
df.fillna(0, inplace=True)

### Now for some actual data science

In [15]:
exclusion = ['SalePrice']
features = [col for col in df.columns if not col in exclusion]
X = df[features]

In [16]:
y = df['SalePrice']

In [17]:
# Calculate baseline
max(y.value_counts(normalize=True))

0.012339585389930898

In [118]:
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [82]:
ss = StandardScaler()
#poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
lasso = Lasso()
#lr = LinearRegression()

In [83]:
# Parameters
params = {
    'lasso__max_iter': [10000],
    #'lasso__alpha': [x * 0.005 for x in range(20)]
    'lasso__alpha': [0.001, 0.01, 0.1, 0.9, 1.0, 6.5, 7.0]
}

In [85]:
ls_pipe = Pipeline([
    ('ss', ss),
#    ('poly', poly),
    ('lasso', lasso)
#    ('lr', lr),
])

In [86]:
gs = GridSearchCV(ls_pipe, param_grid=params, cv=3)

In [79]:
#ls_pipe.fit(X_train, y_train)

In [87]:
gs.fit(X_train, y_train)

/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: Converge

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('ss', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lasso', Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'lasso__max_iter': [10000], 'lasso__alpha': [0.001, 0.01, 0.1, 0.9, 1.0, 6.5, 7.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [81]:
# Within alpha ranges .000 to .095
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

0.7746628843060099
{'lasso__alpha': 6.0, 'lasso__max_iter': 10000}
0.8826288495491688


In [ ]:
# Within alpha ranges .100 to .195
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

In [ ]:
# Within alpha ranges .200 to .295
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

In [ ]:
# Within alpha ranges .300 to .199
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

In [ ]:
# Within alpha ranges .400 to .199
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

In [ ]:
# Within alpha ranges .500 to .199
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

In [ ]:
# Within alpha ranges .600 to .199
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

In [ ]:
# Within alpha ranges .700 to .199
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

In [ ]:
# Within alpha ranges .800 to .199
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

In [30]:
# Within alpha ranges .900 to .199
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

0.7299674188727718
{'lasso__alpha': 0.998, 'lasso__max_iter': 10000}
0.8945183140383304


In [36]:
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

0.729970971648497
{'lasso__alpha': 1, 'lasso__max_iter': 10000}
0.8945262632707403


In [47]:
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

0.7725431775031547
{'lasso__alpha': 1.1, 'lasso__max_iter': 10000}
0.8758687455474201


In [53]:
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

0.7729332398260904
{'lasso__alpha': 1.4, 'lasso__max_iter': 10000}
0.8770775645803374


In [59]:
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

0.7736444709548194
{'lasso__alpha': 2.4, 'lasso__max_iter': 10000}
0.8801623487013762


In [67]:
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

0.7738583581712474
{'lasso__alpha': 3.1, 'lasso__max_iter': 10000}
0.8811175736535973


In [74]:
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

0.774021541321513
{'lasso__alpha': 3.8, 'lasso__max_iter': 10000}
0.8816432509383715


In [88]:
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

0.7749435430410543
{'lasso__alpha': 7.0, 'lasso__max_iter': 10000}
0.8829097824339575


In [119]:
pred = gs.predict(X_test)

In [121]:
mean_squared_error(y_test, pred) ** .5

19564.909755284592

In [ ]:
exclusion = ['SalePrice']
features = [col for col in df.columns if not col in exclusion]
X = df[features]
y = df['SalePrice']

In [127]:
pred = gs.predict(X)

In [128]:
mean_squared_error(y, pred) ** .5

24139.32910812709

In [130]:
sub = pd.DataFrame(data=pred, index=df['Id'])

In [133]:
sub.columns = ['SalePrice']

In [136]:
sub.to_csv('./sub1.csv')

In [138]:
sub.shape

(2051, 1)

In [153]:
test = pd.read_csv('./test.csv')

In [154]:
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [155]:
# Drop columns composed largely of null values. Still on the fence about
# the Misc Features category
test = test.drop(columns=['Alley', 'Pool QC', 'Fence', 'Misc Feature', 'Fireplace Qu'])
# Drop non-numeric columns that are not good dummy candidates
test.drop(columns=['Street', 'Land Contour', 'Functional', 'Land Slope',
                     'Utilities', 'Condition 2', 'Roof Style', 'Roof Matl',
                     'Exterior 2nd', 'Bsmt Cond', 'BsmtFin Type 2', 'Heating',
                     'Electrical', 'Functional', 'Garage Yr Blt', 'Garage Qual',
                     'Garage Cond'], inplace=True)

In [156]:
# Drop remaining columns with low correlation to target before scaling, since
# correlation is independent of scale and origin
test.drop(columns=['Pool Area', 'Screen Porch', '3Ssn Porch', 'Enclosed Porch',
                'Kitchen AbvGr', 'Bedroom AbvGr', 'Bsmt Half Bath',
                'Low Qual Fin SF', 'BsmtFin SF 2', 'PID'], inplace=True)

In [163]:
# Generate dummies with potential valuee
test = pd.get_dummies(test, columns=['Neighborhood', 'MS SubClass',
               'MS Zoning', 'Lot Shape', 'Lot Config', 'Condition 1',
               'Bldg Type', 'House Style','Year Built', 'Year Remod/Add',
               'Exterior 1st', 'Mas Vnr Type', 'Exter Qual', 'Exter Cond',
               'Foundation', 'Bsmt Qual', 'Bsmt Exposure', 'BsmtFin Type 1',
               'Heating QC', 'Central Air', 'Kitchen Qual', 'Garage Type',
               'Garage Finish', 'Paved Drive', 'Mo Sold', 'Yr Sold',
               'Sale Type'], drop_first=True)

In [166]:
test.drop(columns=['Lot Frontage'], inplace=True)

In [167]:
test.fillna(0, inplace=True)

In [175]:
features = [col for col in test.columns]
Xfin = test[features]

In [176]:
Xfin

,Id,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,...,Yr Sold_2010,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD
0,2658,9142,6,8,0.0,0.0,1020.0,1020.0,908,1020,...,0,0,0,0,0,0,0,0,0,1
1,2718,9662,5,4,0.0,0.0,1967.0,1967.0,1967,0,...,0,0,0,0,0,0,0,0,0,1
2,2414,17104,7,5,0.0,554.0,100.0,654.0,664,832,...,0,0,0,0,0,0,1,0,0,0
3,1989,8520,5,6,0.0,0.0,968.0,968.0,968,0,...,0,0,0,0,0,0,0,0,0,1
4,625,9500,6,5,247.0,609.0,785.0,1394.0,1394,0,...,0,0,0,0,0,0,0,0,0,1
5,333,1890,4,6,0.0,294.0,252.0,546.0,546,546,...,1,0,0,0,0,0,0,0,0,1
6,1327,8516,4,6,0.0,0.0,869.0,869.0,1093,0,...,0,0,0,0,0,0,0,0,0,1
7,858,9286,5,7,0.0,196.0,1072.0,1268.0,1268,0,...,0,0,0,0,0,0,0,0,0,1
8,95,3515,7,5,0.0,0.0,840.0,840.0,840,840,...,1,0,0,0,0,0,0,0,0,1
9,1568,10125,6,6,0.0,641.0,276.0,1196.0,1279,0,...,0,0,0,0,0,0,0,0,0,1


In [177]:
pred = gs.predict(Xfin)

ValueError: operands could not be broadcast together with shapes (879,337) (684,) (879,337) 

In [173]:
sub = pd.DataFrame(data=pred, index=test['Id'])

ValueError: Shape of passed values is (1, 406), indices imply (1, 879)